In [ ]:
import torch
import matplotlib.pyplot as plt
import torch
from importlib import reload 
from pathlib import Path 

from unlearning.datasets import gaussian 

from unlearning import probes, leace, plotting
reload(probes)
reload(plotting)
reload(leace)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'



def check_memory():
    print(f"Memory Allocated: {torch.cuda.memory_allocated() / 1024 ** 2} MB")
    print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated() / 1024 ** 2} MB")
    print(f"Memory Cached: {torch.cuda.memory_reserved() / 1024 ** 2} MB")
    print(f"Max Memory Cached: {torch.cuda.max_memory_reserved() / 1024 ** 2} MB")  


BASE_DIR = Path("/n/home04/rrinberg/data_dir__sneel/Lab/rrinberg/results")

# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
# Create a random dataset with 100 samples and 5 features on the selected device
torch.manual_seed(42)

n_samples = 500
n_features = 2
alpha = 2  # Adjust alpha to change the rate of decay
difference = 1.1

# Generate the dataset and obtain the covariance matrix used
X1, cov_matrix = gaussian.generate_powerlaw_gaussian_dataset(n_samples, n_features, alpha=alpha, device=device)
X2, cov_matrix = gaussian.generate_powerlaw_gaussian_dataset(n_samples, n_features, alpha=alpha, device=device, mean = difference)

data = torch.cat((X1, X2), dim=0)

labels = torch.zeros(len(data))
labels[len(X1):] = 1

print(f"X shape: {data.shape}")    
# Whiten the dataset
X_white = leace.whiten_data_torch(data)

# Verify that the covariance of the whitened data is approximately the identity matrix
n_samples = X_white.size(0)
cov_whitened = torch.mm(X_white.T, X_white) / (n_samples - 1)
print("Covariance of the whitened data:\n", cov_whitened)

# Optionally, generate some dummy labels (e.g., for binary classes)
#labels = torch.randint(0, 2, (n_samples,), device=device)

# plot original data with top pcs

plotting.plot_top2_pcs_torch(data, labels=labels)
var_orig = torch.var(data, dim=0)
print(f"Variance of original data: {var_orig}")
# plot whitened data
plotting.plot_top2_pcs_torch(X_white, labels=labels)
var_whitened = torch.var(X_white, dim=0)
covar_whitened = torch.cov(X_white.T)
# round covar_whitened to 2 decimal places
covar_whitened = torch.round(covar_whitened * 100) / 100
print(f"Covariance of whitened data: {covar_whitened}")
print(f"Variance of original data: {var_whitened}")

# plot the covariance matrix of the whitened data
U, S, V = torch.svd(data)
U_whitened, S_whitened, V_whitened = torch.svd(X_white)     
plt.plot(S.cpu().numpy(), label='Original Data')
plt.plot(S_whitened.cpu().numpy(), label='Whitened Data')
plt.xlabel('Index')
plt.ylabel('Singular Value')
plt.title('Singular Values of Original and Whitened Data')
plt.legend()
plt.show()
